# Scraping

스크래핑과 크롤링의 차이  
1.크롤링은 large scale : volumn으로 봤을 때 big data라고 할 수 있음. 크롤러 에이전트가 필요함. 앵커를 찾아서 스크랩터를 계속 돌아다니는 것   
2.스크래핑은 규모가 더 적음 , 필수가 아님, parser가 필요함. 원하는 데이터만 찾아서 정확히 찾아오는 것 

크롤러 > 스크래핑  


아키텍쳐  
멀티 thread : 인터넷으로 부터 웹페이지를 url을 통해서 다운받는다.   


In [1]:
import sqlite3
con = sqlite3.connect("bot1.db")
cur = con.cursor()

In [2]:
import requests
from html import unescape
from html import escape
from bs4 import BeautifulSoup
import bs4
import time
import json
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}

def download(method, url, param=None, data=None, 
             headers = None, maxretries=4 ):
    try :
        resp = requests.request(method, url, params = param, 
                                data=data,headers = headers )
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0 :
            print(maxretries)
            resp = download(method, url, param, data, maxretries - 1)
        else :
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [3]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
html = download("get", url, param={'id':'ppomppu'},headers=headers)
dom = BeautifulSoup(html.text, "lxml")

html.encoding = 'utf-8'

In [4]:
html.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr" />\n<meta http-equiv="Content-Script-Type" content="text/javascript" />\n<meta http-equiv="Content-Style-Type" content="text/css" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="author" content="PPOMPPU CO.">\n<meta name="description" content="�˻�">\n<meta name="keywords" content="">\n\n\n<script async=\'async\' src=\'//www.googletagservices.com/tag/js/gpt.js\'></script>\n<script>var googletag = googletag || {};googletag.cmd = googletag.cmd || [];</script>\n\n<!--\n<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, user-scalable=yes, target-densitydpi=device-dpi" />\n-->\n\n\n<meta property="og:image" content="http://www.ppomppu.co.kr/images/icon_app_20160427.png" />\n\n<meta property="og:site_name" content="�˻�" />\n\n<t

In [5]:
encoding = html.headers["Content-Type"].split('=')[-1]

In [6]:
encoding

'euc-kr'

In [7]:
type(html.content)

bytes

In [8]:
dom.find_all('',{'class':'list_title'})

[<font class="list_title">[11번가] LG 공식판매점 디오스 인덕션 전기레인지 BEF3G 3구 (1,490,000/무료)</font>,
 <font class="list_title">[위메프] EVA 30CM 마사지 폼롤러 (5900/무료)</font>,
 <font class="list_title">[위메프] LG 미니빔 HF60LA FHD 1400루멘 (835,640/무료)</font>,
 <font class="list_title">[11번가] 리스테린 그린티 마일드 750mlx2+250mlx2  (13,500/무료배송)</font>,
 <font class="list_title">[Gsuite] Gsuite Basic India (3700원(월) / 무료)</font>,
 <font class="list_title">[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백</font>,
 <font class="list_title">[옥션] 식사에반하다25개입 (26,530원 / 무배)</font>,
 <font class="list_title">[티몬] (티몬페이전용)해피머니 10만원권 3장(271,500원/쿠폰적용시 269,900원)</font>,
 <font class="list_title">[티몬] 해피머니 10만원권 (90,900원/0원)</font>,
 <font class="list_title">[티몬] 1등급 한돈 삼겹살 600g (6,500/무료배송)</font>,
 <font class="list_title">[우체국쇼핑] 진안 매콤화끈 중독의맛 뼈없는 직화불닭발 1+1(2팩)(8500/무료) 쿠폰적용시7650</font>,
 <font class="list_title">[유튜브] 유튜브프리미엄 인도 ( 2200원(월), 3200원(월)6명 / 무료)</font>,
 <font class="list_title">[11번가] 벨킨 스마트폰 전기종 호환 충전케이블(9,900/무

In [19]:
for tag in dom.select("tr[class^=list]")[1:]:
    tdList = tag.find_all('td',recursive=False)
#    print(len(tdList))
    print(tdList[3].text.strip()) #제목
    print(requests.compat.urljoin(url,tdList[3].a.img['src'])) #이미지
    print('추/비추천수',tdList[5].text.strip()) #추천수
    print('조회수',tdList[6].text.strip(), end="\n\n") #조회수

[11번가] LG 공식판매점 디오스 인덕션 전기레인지 BEF3G 3구 (1,490,000/무료)  9
http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/1/small_323881.jpg?t=20190716150504
추/비추천수 0 - 1
조회수 568

[셀티바] 다이어트유산균(48%할인/무료배송)  6
http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/0/small_323880.jpg?t=20190716151334
추/비추천수 0 - 1
조회수 361

[위메프] EVA 30CM 마사지 폼롤러 (5900/무료)  10
http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/9/small_323879.jpg?t=20190716145420
추/비추천수 
조회수 1992

[위메프] LG 미니빔 HF60LA FHD 1400루멘 (835,640/무료)  2
http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/8/small_323878.jpg?t=20190716145350
추/비추천수 
조회수 934

[11번가] 리스테린 그린티 마일드 750mlx2+250mlx2  (13,500/무료배송)  9
http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/7/small_323877.jpg?t=20190716144040
추/비추천수 
조회수 479

[Gsuite] Gsuite Basic India (3700원(월) / 무료)  48
http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/6/small_323876.jpg?t=20190716134948
추/비추천수 3 - 0
조회수 5500

[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백  27
http://cdn.ppomppu.co.

# 뽐뿌 자유게시판

In [3]:
url = 'http://www.ppomppu.co.kr/zboard/zboard.php'
html = download("get", url, param={'id':'freeboard'},headers=headers)
dom = BeautifulSoup(html.text, "lxml")

In [4]:
links=[]

for tag in dom.select('font.list_title'):
    print(tag.text)
    print(requests.compat.urljoin(url,tag.find_parent()['href']))
    links.append(requests.compat.urljoin(url,tag.find_parent()['href']))

  [알려드립니다] 정치자유게시판 정식 운영에 대해 안내드립니다.
http://www.ppomppu.co.kr/zboard/view.php?id=issue&no=160987
그런데 자살당하려면 부인이 공범이어야
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543247
가족들이 유서를 발견하고 신고해서
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543246
어제자 정두언.swf
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543245
데이터 주기?알려주세요
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543244
요기요 버거킹 매장에서 먹었네요 .jyp
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543243
MB석방해준 판사는 지금 무슨생각을 할까요?
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543242
정두언 이명박이 정계 데뷔 시킴
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543241
변기 쪽 물이 녹물이 나오는건 왜 그런걸까요
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543240
쉽게 무책임하게 자살할 사람이.
http://www.ppomppu.co.kr/zboard/z

In [9]:
links = links[5:]

In [10]:
for link in links:
    html2 = download("get", links[0], headers=headers)
    dom1 = BeautifulSoup(html2.text, "html.parser")
    print(dom1.select_one("table.pic_bg td.han").text)
    for tag in dom1.select("td.han"):
        print(tag.text)
    break;


MB 돌아 다니라고
 
석방해준 판사는
 
정의원님 자살 소식을 듣고
 
지금 무슨 생각을 하고 있을까요?
 
정말, 물어보고 싶네요...
 
무슨 생각인지....


MB석방해준 판사는 지금 무슨생각을 할까요? 2




이메일
북마크
밴 드
트위터


페이스북
카카오스토리




주소복사





이름: 

MB 돌아 다니라고
 
석방해준 판사는
 
정의원님 자살 소식을 듣고
 
지금 무슨 생각을 하고 있을까요?
 
정말, 물어보고 싶네요...
 
무슨 생각인지....


In [14]:
for link in links:
    html2 = download("get", links[0], headers=headers)
    dom1 = BeautifulSoup(html2.text, "html.parser")
    print('내용: ')
    print(dom1.select_one("table.pic_bg td.han").text)
    
    reply = dom1.select('#quote .info_bg .han')
   
    if len(reply) >0:
        for tag in reply:
            print('reply:')
            print(tag.text.strip(), end='\n\n')
            print()
        break;


내용: 

MB 돌아 다니라고
 
석방해준 판사는
 
정의원님 자살 소식을 듣고
 
지금 무슨 생각을 하고 있을까요?
 
정말, 물어보고 싶네요...
 
무슨 생각인지....
reply:
아...살았다......
 
정도?


reply:
다음은 내 차례인가....


reply:
더더더 충성해야�떪�


